
<span style="font-size:3em;"> Elastic DFT</span>
## Calculating Elastic Constants with Quantum Espresso
Developed by Robert J. Appleton, Chukwuma Ezenwata, Ching-Chien Chen, Kat Nykiel, Saswat Mishra, and Alejandro Strachan

## Integrating with Cell Relax DFT database
In this tool we query relaxed structures from the Cell Relax DFT tool (see [here](https://nanohub.org/tools/cellrelaxdft)), perform deformations based on a set of strain/shear magnitudes, and then compute the corresponding stress tensor using density functional theory (DFT). The DFT code utilized in this tool is [Quantum Espresso](10.1088/0953-8984/21/39/395502) and the calculation parameters are adopted to match the level of theory and convergence criteria used in the relaxation. In order to run this tool on a material, there must exist a relaxation run in the Cell Relax DFT database. 

## Fitting the Elastic Tensor
The elastic tensor is fit using the [pymatgen elasticity package](https://pymatgen.org/pymatgen.analysis.elasticity.html) and specifically the Moore-Penrose pseudo-inverse method.


In [ ]:
# Import simtool libraries
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

In [ ]:
simToolName = "elasticdft"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print("%18s = %s" % (key,simToolLocation[key]))

In [ ]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

In [ ]:
# get the list of inputs
inputs = getSimToolInputs(simToolLocation)
print(inputs)

In [ ]:
# get the list of outputs
outputs = getSimToolOutputs(simToolLocation)
print(outputs)

In [ ]:
mp_id = "mp-8062"
squid_id = "cellrelaxdft/r67/fd70c16f2adda1e1377b61ccf9dee2d378bec539"

In [ ]:
strains = [-0.01,-0.005,0.005,0.01]
shears = [-0.02,-0.01,0.01,0.02]

In [ ]:
inputs['mp_id'].value = mp_id
inputs['squid_id'].value = squid_id
inputs['strains'].value = strains
inputs['shears'].value = shears

In [ ]:
inputs

In [ ]:
r = Run(simToolLocation,inputs)

In [ ]:
r.getResultSummary()